In [5]:

from google.oauth2.service_account import Credentials
import openai
import httplib2
from googleapiclient.discovery import build
from google.oauth2 import service_account

In [2]:

# Dictionary to store the placeholder replacements
placeholder_replacements = {
    'name': "Sulemaan Farooq",
    'email': "shf46@rutgers.edu",
    'phone': "+1234567890",
    'city': "Cityville",
    'state': "Stateville",
    'linkedin_link': "https://www.linkedin.com/in/sulemaanfarooq/",
    # 'resume_link': "https://www.example.com/resume",
    # 'github_link': "https://github.com/sulemaan",
    'university': "University of Example",
    'university start date': "September 2018",
    'university end date': "May 2022",
    'major': "Computer Science",
    'minor': "Business Administration",
    'spoken_languages': "English, Spanish",
    'languages': "Python, Java, JavaScript",
    'technologies': "React, Django, MongoDB",
    'leadership': "President, Computer Science Club",
    'degree_type': "Bachelor of Science",
}

placeholder_replacements.update({
    'experience1': "Software Developer Intern",
    'title1': "ABC Tech Solutions",
    'experience1 start date': "June 2021",
    'experience1 end date': "August 2021",
    'experience1 location': "Cityville, State",
    'experience1 bullet1': "Developed and maintained web applications using Django and React.",
    'experience1 bullet2': "Collaborated with cross-functional teams to deliver high-quality software.",
    'experience1 bullet3': "Participated in daily stand-up meetings and sprint planning sessions.",
})

placeholder_replacements.update({
    'project1': "E-commerce Website",
    'skills1': "React, Node.js, MongoDB",
    'project1 bullet1': "Designed and implemented the front-end of an e-commerce website using React.",
    'project1 bullet2': "Developed RESTful APIs and database models using Node.js and MongoDB.",
})

# Path to your service account credentials JSON file
SERVICE_ACCOUNT_FILE = '/Users/cheesenaan/Documents/projects/resume_app/project/.ipynb_checkpoints/resume_App/resume_app/resume_app/doc.json'
#SERVICE_ACCOUNT_FILE = '/home/displayai/displayai/resume_app/doc.json'

# ID of the Google Doc you want to modify
DOCUMENT_ID = '1PVKqAkOTjdorBmlJOIGHYhIjYuBPxuPZzKeN4TUQZNE'

# Folder ID where the copy will be placed
FOLDER_ID = '1aGcF78a65Nus-K9kCv8P7NzMBSZjJnNY'

# Authenticate and create the Google Drive API service
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=credentials)

# Create a copy of the document
copy_metadata = {
    'name':  'Sulemaan - Resume',
    'parents': [FOLDER_ID]
}
copy_response = service.files().copy(fileId=DOCUMENT_ID,  body=copy_metadata).execute()
copy_id = copy_response['id']

print(f"Copy created with ID: {copy_id}")

DOCUMENT_ID = str(copy_id)


# Authenticate and create the Google Docs API service
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/documents'])
service = build('docs', 'v1', credentials=credentials)


# Get the content of the Google Doc
doc = service.documents().get(documentId=DOCUMENT_ID).execute()

# Process each content element and replace placeholders
for content in doc['body']['content']:
    if 'paragraph' in content:
        elements = content['paragraph']['elements']
        for element in elements:
            if 'textRun' in element:
                text_run = element['textRun']
                if 'content' in text_run:
                    content_text = text_run['content']
                    for placeholder, replacement in placeholder_replacements.items():
                        if placeholder in content_text:
                            content_text = content_text.replace(placeholder, replacement)
                            text_run['content'] = content_text

# Create the requests list for batch updating
requests = []
for placeholder, replacement in placeholder_replacements.items():
    requests.append({
        'replaceAllText': {
            'containsText': {
                'text': '{{' + placeholder + '}}',
                'matchCase': False
            },
            'replaceText': replacement
        }
    })

# Execute the batch update requests
result = service.documents().batchUpdate(documentId=DOCUMENT_ID,  body={'requests': requests}).execute()

print('Placeholders replaced successfully in the Google Doc')
# Get the content of the Google Doc
doc = service.documents().get(documentId=DOCUMENT_ID).execute()
body_content = doc['body']['content']

# Iterate over the paragraphs in reverse order
requests = []
for i in range(len(body_content) - 1, -1, -1):
    paragraph = body_content[i]
    if 'paragraph' in paragraph:
        elements = paragraph['paragraph']['elements']
        contains_template_text = any('{{' in element['textRun']['content'] and '}}' in element['textRun']['content'] for element in elements)
        if contains_template_text:
            # Remove the paragraph if it contains {{}} template text
            requests.append({
                'deleteContentRange': {
                    'range': {
                        'startIndex': paragraph['startIndex'],
                        'endIndex': paragraph['endIndex']
                    }
                }
            })

# Execute the batch update requests to remove paragraphs with unfilled {{}} template text
if requests:
    service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': requests}).execute()
    print("Lines with unfilled {{}} template text removed successfully!")


def execute_remove_blank_lines():
    # Execute the removeBlankLines function in the Google Doc
    requests = [
        {
            'runFunction': {
                'function': 'removeEmptyLines',
            },
        }
    ]

    result = service.documents().batchUpdate(documentId=DOCUMENT_ID, body={'requests': requests}).execute()

    print('removeBlankLines function executed successfully!')


# Get the link of the Google Doc
resume_link = f"https://docs.google.com/document/d/{DOCUMENT_ID}"
print("Link to the Google Doc:", resume_link)


Copy created with ID: 15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg
Placeholders replaced successfully in the Google Doc
Lines with unfilled {{}} template text removed successfully!
Link to the Google Doc: https://docs.google.com/document/d/15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg


In [30]:
def remove_blank_lines(document_id):
    # Get the content of the Google Doc
    doc = service.documents().get(documentId=document_id).execute()
    body = doc['body']

    # List to store delete requests
    delete_rows = []

    # Iterate through all elements in reverse order
    total_elements = len(body['content'])
    i = total_elements - 1
    while i >= 0:
        element = body['content'][i]

        # Check if the element is a paragraph
        if 'paragraph' in element:
            paragraph = element['paragraph']
            text = paragraph['elements'][0]['textRun']['content'].strip()

            # Check if the current element is empty and the previous element exists and is also empty
            if i != total_elements - 1 and len(text) == 0 and 'paragraph' in body['content'][i - 1] and \
                    body['content'][i - 1]['paragraph']['elements'][0]['textRun']['content'].strip() == '':
                delete_rows.append(i)

        i -= 1

    result = service.documents().batchUpdate(
    documentId=DOCUMENT_ID, body={'requests': requests}).execute()


# Call the remove_blank_lines function with your document ID
remove_blank_lines('15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg')


In [33]:
def remove_blank_lines(document_id):
    # Get the content of the Google Doc
    doc = service.documents().get(documentId=document_id).execute()
    body = doc['body']

    # List to store delete requests
    delete_start_indices = []
    delete_end_indices = []
    delete_rows = []

    # Iterate through all elements in reverse order
    total_elements = len(body['content'])
    i = total_elements - 1
    while i >= 0:
        element = body['content'][i]

        # Check if the element is a paragraph
        if 'paragraph' in element:
            paragraph = element['paragraph']
            text = paragraph['elements'][0]['textRun']['content'].strip()

            # Check if the current element is empty and the previous element exists and is also empty
            if i != total_elements - 1 and len(text) == 0 and 'paragraph' in body['content'][i - 1] and \
                    body['content'][i - 1]['paragraph']['elements'][0]['textRun']['content'].strip() == '':
                # Store the character index where the row starts
                start_index = paragraph['elements'][0]['startIndex']
                end_index = paragraph['elements'][0]['endIndex']
                delete_start_indices.append(start_index)
                delete_end_indices.append(end_index)
                delete_rows.append(i)

        i -= 1

    print(delete_rows)
    print(delete_start_indices)
    print(delete_end_indices)

    requests = [
        {
            'deleteContentRange': {
                'range': {
                    'startIndex': delete_start_indices[0],
                    'endIndex': delete_end_indices[0],
                }

            }

        },
    ]
    result = service.documents().batchUpdate(
        documentId=DOCUMENT_ID, body={'requests': requests}).execute()





# Call the remove_blank_lines function with your document ID
remove_blank_lines('15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg')


[25, 24, 23, 22, 16, 15, 14]
[918, 917, 916, 915, 725, 724, 723]
[919, 918, 917, 916, 726, 725, 724]


In [34]:
def remove_blank_lines(document_id):
    # Get the content of the Google Doc
    doc = service.documents().get(documentId=document_id).execute()
    body = doc['body']

    # Lists to store delete requests
    delete_start_indices = []
    delete_end_indices = []
    delete_rows = []

    # Iterate through all elements in reverse order
    total_elements = len(body['content'])
    i = total_elements - 1
    while i >= 0:
        element = body['content'][i]

        # Check if the element is a paragraph
        if 'paragraph' in element:
            paragraph = element['paragraph']
            text = paragraph['elements'][0]['textRun']['content'].strip()

            # Check if the current element is empty and the previous element exists and is also empty
            if i != total_elements - 1 and len(text) == 0 and 'paragraph' in body['content'][i - 1] and \
                    body['content'][i - 1]['paragraph']['elements'][0]['textRun']['content'].strip() == '':
                # Store the character index where the row starts
                start_index = paragraph['elements'][0]['startIndex']
                end_index = paragraph['elements'][0]['endIndex']
                delete_start_indices.append(start_index)
                delete_end_indices.append(end_index)
                delete_rows.append(i)

        i -= 1

    # Create a list of delete requests based on the stored start and end indices
    requests = [
        {
            'deleteContentRange': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': end_index,
                }
            }
        }
        for start_index, end_index in zip(delete_start_indices, delete_end_indices)
    ]

    # Execute the batch update with delete requests
    result = service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

# Call the remove_blank_lines function with your document ID
remove_blank_lines('15SavP_3CQrRMn99_II5Iz9idc5ob94Zge5KnI4uqRVg')


In [3]:
import stripe


stripe.api_key = 'sk_test_51OibSIBFOKaICuMNwUyc27HPurXFcLQqowTzFRqr7RUcw0tbGu4WR4XkFRY3qEqN4a5bb5qX93ApQzmvjT7Ew6o200MbPOzO5i'

checkout_session = stripe.checkout.Session.create(
payment_method_types=['card', 'cashapp', 'us_bank_account'],
line_items=[
    {
        'price': 'price_1OibgCBFOKaICuMNShWHaBHo',
        'quantity': 1,
    },
],
mode='subscription',
# customer_creation='always',
success_url= 'http://127.0.0.1:8000//payment_successful?session_id={CHECKOUT_SESSION_ID}',
cancel_url=  'http://127.0.0.1:8000//payment_cancelled',
)
print(checkout_session)

{
  "after_expiration": null,
  "allow_promotion_codes": null,
  "amount_subtotal": 999,
  "amount_total": 999,
  "automatic_tax": {
    "enabled": false,
    "liability": null,
    "status": null
  },
  "billing_address_collection": null,
  "cancel_url": "http://127.0.0.1:8000//payment_cancelled",
  "client_reference_id": null,
  "client_secret": null,
  "consent": null,
  "consent_collection": null,
  "created": 1708963791,
  "currency": "usd",
  "currency_conversion": null,
  "custom_fields": [],
  "custom_text": {
    "after_submit": null,
    "shipping_address": null,
    "submit": null,
    "terms_of_service_acceptance": null
  },
  "customer": null,
  "customer_creation": "always",
  "customer_details": null,
  "customer_email": null,
  "expires_at": 1709050191,
  "id": "cs_test_a1yjzgRas5F16J2BzDgLWLeaeY9WAhc7pzLuFy7svfSvl6pvZbHPzViDTu",
  "invoice": null,
  "invoice_creation": null,
  "livemode": false,
  "locale": null,
  "metadata": {},
  "mode": "subscription",
  "object": 

In [4]:
print("stripe.checkout.Session.retrieve is ")
print(stripe.checkout.Session.retrieve(checkout_session.id))

stripe.checkout.Session.retrieve is 
{
  "after_expiration": null,
  "allow_promotion_codes": null,
  "amount_subtotal": 999,
  "amount_total": 999,
  "automatic_tax": {
    "enabled": false,
    "liability": null,
    "status": null
  },
  "billing_address_collection": null,
  "cancel_url": "http://127.0.0.1:8000//payment_cancelled",
  "client_reference_id": null,
  "client_secret": null,
  "consent": null,
  "consent_collection": null,
  "created": 1708963791,
  "currency": "usd",
  "currency_conversion": null,
  "custom_fields": [],
  "custom_text": {
    "after_submit": null,
    "shipping_address": null,
    "submit": null,
    "terms_of_service_acceptance": null
  },
  "customer": null,
  "customer_creation": "always",
  "customer_details": null,
  "customer_email": null,
  "expires_at": 1709050191,
  "id": "cs_test_a1yjzgRas5F16J2BzDgLWLeaeY9WAhc7pzLuFy7svfSvl6pvZbHPzViDTu",
  "invoice": null,
  "invoice_creation": null,
  "livemode": false,
  "locale": null,
  "metadata": {},
 

In [5]:
print("stripe.checkout.Session.retrieve is ")
print(stripe.checkout.Session.retrieve(checkout_session.id))

stripe.checkout.Session.retrieve is 
{
  "after_expiration": null,
  "allow_promotion_codes": null,
  "amount_subtotal": 999,
  "amount_total": 999,
  "automatic_tax": {
    "enabled": false,
    "liability": null,
    "status": null
  },
  "billing_address_collection": null,
  "cancel_url": "http://127.0.0.1:8000//payment_cancelled",
  "client_reference_id": null,
  "client_secret": null,
  "consent": null,
  "consent_collection": null,
  "created": 1708963791,
  "currency": "usd",
  "currency_conversion": null,
  "custom_fields": [],
  "custom_text": {
    "after_submit": null,
    "shipping_address": null,
    "submit": null,
    "terms_of_service_acceptance": null
  },
  "customer": "cus_PdNWtHmmDQTfNM",
  "customer_creation": "always",
  "customer_details": {
    "address": {
      "city": null,
      "country": "QA",
      "line1": null,
      "line2": null,
      "postal_code": null,
      "state": null
    },
    "email": "test@gmail.com",
    "name": "Stripe Test",
    "phone"

code to retrieve stripe subscription unique value

In [9]:
stripe_payment_data = stripe.checkout.Session.retrieve(checkout_session.id)
subscription_value = stripe_payment_data["subscription"]
subscription_value

'sub_1Oo6laBFOKaICuMNeaUGMKAz'

code to cancel a subscription given the subscription value. current month is not cancelled. good to the end of the month

You can reactivate subscriptions scheduled for cancellation (with the cancel_at_period_end parameter) by updating cancel_at_period_end to false. You can reactivate the subscription at any point up to the end of the period.



In [6]:

stripe.Subscription.modify(
  "sub_1Oo6laBFOKaICuMNeaUGMKAz",
  cancel_at_period_end=True,
)


<Subscription subscription id=sub_1Oo6laBFOKaICuMNeaUGMKAz at 0x7fb9d86e0f90> JSON: {
  "application": null,
  "application_fee_percent": null,
  "automatic_tax": {
    "enabled": false,
    "liability": null
  },
  "billing_cycle_anchor": 1708963858,
  "billing_cycle_anchor_config": null,
  "billing_thresholds": null,
  "cancel_at": 1711469458,
  "cancel_at_period_end": true,
  "canceled_at": 1708964060,
  "cancellation_details": {
    "comment": null,
    "feedback": null,
    "reason": "cancellation_requested"
  },
  "collection_method": "charge_automatically",
  "created": 1708963858,
  "currency": "usd",
  "current_period_end": 1711469458,
  "current_period_start": 1708963858,
  "customer": "cus_PdNWtHmmDQTfNM",
  "days_until_due": null,
  "default_payment_method": "pm_1Oo6lZBFOKaICuMNrZuPjMOS",
  "default_source": null,
  "default_tax_rates": [],
  "description": null,
  "discount": null,
  "ended_at": null,
  "id": "sub_1Oo6laBFOKaICuMNeaUGMKAz",
  "invoice_settings": {
    "acc

get start date and end date of subscription

In [21]:
from datetime import datetime
import stripe


subscription_id = "sub_1Oo6laBFOKaICuMNeaUGMKAz"
subscription_id = "sub_1Oo8dvBFOKaICuMNNQwLJET0"

# Retrieve subscription information from Stripe
subscription = stripe.Subscription.retrieve(subscription_id)

# Extract start and end dates (assuming they are Unix timestamps)
start_timestamp = subscription.current_period_start
end_timestamp = subscription.current_period_end

# Convert timestamps to datetime objects
start_datetime = datetime.utcfromtimestamp(start_timestamp)
end_datetime = datetime.utcfromtimestamp(end_timestamp)

# Format datetime objects as strings
start_date_readable = start_datetime.strftime('%Y-%m-%d %H:%M:%S UTC')
end_date_readable = end_datetime.strftime('%Y-%m-%d %H:%M:%S UTC')

# Print the readable dates
print("Subscription Start Date:", start_date_readable)
print("Subscription End Date:", end_date_readable)


Subscription Start Date: 2024-02-26 18:11:11 UTC
Subscription End Date: 2024-03-26 18:11:11 UTC


Extract the subscription status

In [22]:
# Extract the subscription status
subscription_status = subscription.status

# Print the subscription status
print("Subscription Status:", subscription_status)

Subscription Status: active


check if a Stripe subscription is going to be canceled

In [23]:
# Check if the subscription is scheduled to be canceled
if subscription.cancel_at is not None:
    print("Subscription is scheduled to be canceled.")
    cancel_at_date = datetime.utcfromtimestamp(subscription.cancel_at).strftime('%Y-%m-%d %H:%M:%S UTC')
    print("Cancellation Date:", cancel_at_date)
else:
    print("Subscription is not scheduled to be canceled.")

Subscription is not scheduled to be canceled.


In [32]:
# Extract and print other basic details
from datetime import datetime
import stripe

stripe.api_key = 'sk_test_51OibSIBFOKaICuMNwUyc27HPurXFcLQqowTzFRqr7RUcw0tbGu4WR4XkFRY3qEqN4a5bb5qX93ApQzmvjT7Ew6o200MbPOzO5i'


subscription_id = "sub_1Oo6laBFOKaICuMNeaUGMKAz"

# Retrieve subscription information from Stripe
subscription = stripe.Subscription.retrieve(subscription_id)


# Extract and print other basic details
print("Subscription ID:", subscription.id)
print("Customer ID:", subscription.customer)

# Access the items associated with the subscription
items = subscription.get("items")

# Print the items information
print("Subscription Items:")
for item in items.get("data", []):
    print("  - Item ID:", item.id)
    print("    Plan ID:", item.plan.id)
    print("    Quantity:", item.quantity)
    print("    Price ID:", item.price.id)
    print("    Price ID:", item.price.unit_amount)
    print("    Price ID:", item.price.product)



Subscription ID: sub_1Oo6laBFOKaICuMNeaUGMKAz
Customer ID: cus_PdNWtHmmDQTfNM
Subscription Items:
  - Item ID: si_PdNWacfWgdtiSV
    Plan ID: price_1OibgCBFOKaICuMNShWHaBHo
    Quantity: 1
    Price ID: price_1OibgCBFOKaICuMNShWHaBHo
    Price ID: 999
    Price ID: prod_PXh4cOBIbMFV6u


In [17]:
price_id = 'price_1OibgCBFOKaICuMNShWHaBHo'

# Retrieve price information from Stripe
price = stripe.Price.retrieve(price_id)

# Access the amount and convert it to dollars
amount = float(price.unit_amount_decimal) / 100  # Convert from cents to dollars

# Retrieve product information if available
product_name = None
if price.product:
    product = stripe.Product.retrieve(price.product)
    product_name = product.name

# Print the information
print("Price Amount: $", amount)
print("Product Name:", product_name)

Price Amount: $ 9.99
Product Name: Basic


In [3]:
import stripe

stripe.api_key = 'sk_test_51OibSIBFOKaICuMNwUyc27HPurXFcLQqowTzFRqr7RUcw0tbGu4WR4XkFRY3qEqN4a5bb5qX93ApQzmvjT7Ew6o200MbPOzO5i'
subscription_id = "sub_1Oo9eCBFOKaICuMN3A91SYnx"

# Retrieve subscription information from Stripe
subscription = stripe.Subscription.retrieve(subscription_id)

# Retrieve customer information using the customer ID from the subscription
customer_id = subscription.customer
customer = stripe.Customer.retrieve(customer_id)

# Get the customer email
customer_email = customer.email
print("Customer Email:", customer_email)


Customer Email: economy@gmail.com


ModuleNotFoundError: No module named 'your_project'